In [31]:
import numpy as np
import sympy as sp

Global definitions

In [32]:
# Number of states in the individual Hamiltonian
N = 4

# I store in a dictionary the equivalence between the
# total state and the occupancy notation
states = dict()
states[0] = [0, 0, 0, 0]
states[1] = [1, 0, 0, 0]
states[2] = [0, 1, 0, 0]
states[3] = [0, 0, 1, 0]
states[4] = [0, 0, 0, 1]
states[5] = [1, 1, 0, 0]
states[6] = [1, 0, 1, 0]
states[7] = [1, 0, 0, 1]
states[8] = [0, 1, 1, 0]
states[9] = [0, 1, 0, 1]
states[10] = [0, 0, 1, 1]
states[11] = [1, 1, 1, 0]
states[12] = [1, 1, 0, 1]
states[13] = [1, 0, 1, 1]
states[14] = [0, 1, 1, 1]
states[15] = [1, 1, 1, 1]

Helper functions

In [33]:
# This function gives you the state number for a given 
# occupancy state. For example: getState([1,1,0,0]) 
# returns 5
def getState(a):
    for s, o in states.items():
        sameState = True
        for i, j in enumerate(a):
            if j != o[i]:
                sameState= False
        if sameState:
            return s

# This function returns a vector associated to the i
# state. For example: getStateGlobalBasis(1) =
# [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def getStateGlobalBasis(i):

    s = np.zeros(N**2)
    s[i] = 1
    return s

In [34]:
# getState([0, 1, 1, 0])

In [35]:
# getStateGlobalBasis(2)

one-two body matrix

In [36]:
L = 1

# Wavefunction Solutions
x = sp.Symbol('x')
x_1 = sp.Symbol('x_1')

def well_wavefunction(n, L, x = sp.Symbol('x')):
    """
    Calcula la función de onda para un estado dado (n) en un pozo de potencial infinito.
    
    Parámetros:
        n (int): Estado cuántico del electrón.
    
    Retorna:
        La función de onda.
    """
    
    # Factor de normalización
    normalization_factor = sp.sqrt(2/L)
    
    # Función de onda radial
    well_wavefunction = normalization_factor * sp.sin(n*sp.pi*x/L)
    
    return well_wavefunction


for n in range(2*2):
    globals()[f'phi_{n + 1}'] = well_wavefunction(n//2 + 1, L)
    globals()[f'dd_phi_{n + 1}'] = sp.diff(globals()[f'phi_{n + 1}'], x, x)

for n in range(2*2):
    globals()[f'phi_{n + 1}_1'] = well_wavefunction(n//2 + 1, L, sp.Symbol('x_1'))
    globals()[f'phi_{n + 1}_2'] = well_wavefunction(n//2 + 1, L, sp.Symbol('x_1'))


def one_body(L = 1, N = 3, v_0 = 1):
    h_pq = []

    for i in range(1, 2*N+1):
        h_pq.append([])
        for j in range(1, 2*N+1):
            if i == j:
                globals()[f'h_{i}{j}'] = -1/2 * sp.integrate(globals()[f'phi_{i}']*globals()[f'dd_phi_{j}'], (x, 0, L))
            else:
                globals()[f'h_{i}{j}'] = 0
                
            h_pq[i-1].append(globals()[f'h_{i}{j}'])

    size = len(h_pq)
    h1_a = np.array(h_pq, dtype=float)

    return size, h1_a


import itertools

def two_body(size, L, N = 3, v_0 = 1):

    rhpq = []
    for i in range(0, 4):
        rhpq.append([1+i, 1+i])
        rhpq.append([1+i, 1+(i+2)%4])

    rhrs = []
    for i in range(0, 4):
        rhrs.append([1+i, 1+i])
        rhrs.append([1+i, 1+(i+2)%4])


    lista = []
    for i in itertools.product(rhpq, rhrs):
        j = i[0] + i[1]
        lista.append(j)


    h_ijkl = np.zeros((size, size, size, size))

    for i,l,j,k in lista:
        globals()[f'h_{i}{j}{k}{l}'] = v_0*\
                                       globals()[f'phi_{i}_1']*globals()[f'phi_{j}_2']*\
                                       globals()[f'phi_{k}_2']*globals()[f'phi_{l}_1']
        
        print(i, j, k, l)

        h_ijkl[i-1][j-1][k-1][l-1] = sp.integrate(globals()[f'h_{i}{j}{k}{l}'], (x_1, 0, L))


    h2_aa = np.array(h_ijkl, dtype=float)

    return h2_aa

In [37]:
size, hpq = one_body(L, 2)

In [39]:
gpqrs = two_body(size, L, 2, 1)

1 1 1 1
1 1 3 1
1 2 2 1
1 2 4 1
1 3 3 1
1 3 1 1
1 4 4 1
1 4 2 1
1 1 1 3
1 1 3 3
1 2 2 3
1 2 4 3
1 3 3 3
1 3 1 3
1 4 4 3
1 4 2 3
2 1 1 2
2 1 3 2
2 2 2 2
2 2 4 2
2 3 3 2
2 3 1 2
2 4 4 2
2 4 2 2
2 1 1 4
2 1 3 4
2 2 2 4
2 2 4 4
2 3 3 4
2 3 1 4
2 4 4 4
2 4 2 4
3 1 1 3
3 1 3 3
3 2 2 3
3 2 4 3
3 3 3 3
3 3 1 3
3 4 4 3
3 4 2 3
3 1 1 1
3 1 3 1
3 2 2 1
3 2 4 1
3 3 3 1
3 3 1 1
3 4 4 1
3 4 2 1
4 1 1 4
4 1 3 4
4 2 2 4
4 2 4 4
4 3 3 4
4 3 1 4
4 4 4 4
4 4 2 4
4 1 1 2
4 1 3 2
4 2 2 2
4 2 4 2
4 3 3 2
4 3 1 2
4 4 4 2
4 4 2 2


In [40]:
# for i in range(4):
#     for j in range(4):
        
#         print(i+1, j+1)
#         print(gpqrs[i, j])

In [41]:
gpqrs

array([[[[1.5, 0. , 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 1. , 0. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [1.5, 0. , 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 1. , 0. ]],

        [[0. , 0. , 1. , 0. ],
         [0. , 0. , 0. , 0. ],
         [1. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [0. , 0. , 1. , 0. ],
         [0. , 0. , 0. , 0. ],
         [1. , 0. , 0. , 0. ]]],


       [[[0. , 1.5, 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 1. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [0. , 1.5, 0. , 0. ],
         [0. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 1. ]],

        [[0. , 0. , 0. , 1. ],
         [0. , 0. , 0. , 0. ],
         [0. , 1. , 0. , 0. ],
         [0. , 0. , 0. , 0. ]],

        [[0. , 0. , 0. , 0. ],
         [0. , 0. , 0. , 1. ],
         [0. , 0. , 0. , 0. ],
         [0. , 1. , 0. 

Matrix calculations

In [42]:
# Applys the operator A+j on the state 'state'
def applyAp(j, state):

    newstatevector = states[state].copy()
    #Estimate the sign which is the sum of the numbers of 1 before j
    sumones = 0
    for i in range(0, j):
        sumones = sumones + states[state][i]
    sign = 1
    if sumones % 2 != 0:
        sign = -1

    if states[state][j] == 0:
        newstatevector[j] = 1
        newstate = getState(newstatevector)
    else:
        sign = 0
        newstate = 0
    vector = np.zeros(N**2, dtype=np.int32)
    if sign == 0:
        return vector
    else:
        vector[newstate] = 1
        return sign * vector
    return vector

In [43]:
# applyAp(0, 0)

In [44]:
# Applys the operator Aj on the state 'state'
def applyA(j, state):

    newstatevector = states[state].copy()
    #Estimate the sign which is the sum of the numbers of 1 before j
    sumones = 0
    for i in range(0, j):
        sumones = sumones + states[state][i]
    sign = 1
    if sumones % 2 != 0:
        sign = -1

    if states[state][j] == 1:
        newstatevector[j] = 0
        newstate = getState(newstatevector)
    else:
        sign = 0
        newstate = 0
    vector = np.zeros(N**2, dtype=np.int32)
    if sign == 0:
        return vector
    else:
        vector[newstate] = 1
        return sign * vector
    return vector

In [45]:
# applyA(3, 14)

In [46]:
# applyA(0, 14)

In [47]:
# Builds the full A+j matrix
def makeMatrixAp(j):

    mat = applyAp(j, 0)
    for i in range(1, N**2):
        mat = np.vstack((mat, applyAp(j, i)))
    return mat.T

In [48]:
# makeMatrixAp(0)

In [49]:
# Builds the full A+j matrix
def makeMatrixA(j):

    mat = applyA(j, 0)
    for i in range(1, N**2):
        mat = np.vstack((mat, applyA(j, i)))
    return mat.T

In [50]:
# makeMatrixA(0)

In [51]:
# Builds the full single particle Hamiltonian
def buildSingleParticleH(h, A, Ap):

    H = np.zeros((N*N, N*N))
    for i in range(0, N):
        for j in range(0, N):
            H = H + h[i, j] * np.matmul(Ap[i], A[j])

    return H


# Builds the single particle Hamiltonian restricted to
# the interesting states given as a vector of states
def buildSingleParticleHRestricted(H, interestingStates):

    n = len(interestingStates)
    
    Hres = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            statei = interestingStates[i]
            statej = interestingStates[j]
            si = getStateGlobalBasis(statei)
            sj = getStateGlobalBasis(statej)
            Hres[i, j] = np.matmul(si, np.matmul(H, sj))

    return Hres


# Builds the full double particle Hamiltonian
def buildDoubleParticleH(h, A, Ap):

    H = np.zeros((N*N, N*N))
    for i in range(0, N):
        for j in range(0, N):
            for k in range(0, N):
                for l in range(0, N):
                    # print(i, j, k, l)
                    # print(np.matmul(Ap[i], np.matmul(Ap[j], np.matmul(A[k], A[l]))))
                    H = H + h[i, j, k, l] * np.matmul(np.matmul(Ap[i], Ap[j]), np.matmul(A[k], A[l]))

    return H


# Creates a vector with the A and A+ operators
def createOperators():

    A = []
    Ap = []
    for i in range(0, N):
        A.append(makeMatrixA(i))
        Ap.append(makeMatrixAp(i))
    return A, Ap

Code

In [52]:
#We create the operators 
A, Ap = createOperators()

#We create the hpq matrix
hpq = np.zeros((N,N))
hpq[0, 0] = 1.
hpq[1, 1] = 1.
hpq[2, 2] = 5.
hpq[3, 3] = 5.


#We create the hpqrs matrix
hpqrs = gpqrs


#We build the full single particle Hamiltonian
Hsingle = buildSingleParticleH(hpq, A, Ap)

#We build the full double particle Hamiltonian
Hdouble = buildDoubleParticleH(hpqrs, A, Ap)

#H = Hsingle + Hdouble
H = Hsingle + Hdouble

#We restrict to the states in which we are interested in
interestingStates = [5, 6, 7, 8, 9, 10]
Hres = buildSingleParticleHRestricted(H, interestingStates)

print(Hres)

[[ 5.  0.  0.  0.  0.  2.]
 [ 0.  6.  0.  0.  0.  0.]
 [ 0.  0.  8. -2.  0.  0.]
 [ 0.  0. -2.  8.  0.  0.]
 [ 0.  0.  0.  0.  6.  0.]
 [ 2.  0.  0.  0.  0. 13.]]
